# 猫狗大战

kaggle题目 [Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats)

设计一种算法，区分图片中是猫还是狗。

## 一、引入函数库

- numpy: Anaconda环境自带
- tensorflow: 自行下载，gpu版
- cPickle: Anaconda环境自带
- cv2: opencv2
- datetime: Anaconda环境自带
- sklearn.model_selection.train_test_split: Anaconda环境自带，需要高版本sklearn

In [ ]:
import tensorflow as tf
import numpy as np
import cPickle as pickle
import cv2
import datetime
from sklearn.model_selection import train_test_split

## 二、读取数据

In [ ]:
train, train_label = pickle.load(open('data_process/train_image_label_list.p', mode='rb'))
test = train[(len(train)-1000):]
test_label = train_label[(len(train_label)-1000):]
train = train[:(len(train)-1000)]
train_label = train_label[:(len(train_label)-1000)]

## 三、神经网络构建

### 3.1 参数设置

In [ ]:
N_CLASSES = 2
BATCH_SIZE = 16
CAPACITY = 2000
IMG_W = 208
IMG_H = 208
EPOCH_NUMS = 10
learning_rate = 0.0001

### 3.2 神经网络网络

In [ ]:
def inference(images, batch_size, n_classes):
    with tf.variable_scope('conv1') as scope:
        weights = tf.get_variable('weights',
                                  shape = [3, 3, 3, 16],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.1, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                  shape = [16],
                                  dtype = tf.float32,
                                  initializer = tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(images, weights, strides=[1,1,1,1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        
    # pool1 and norm1
    with tf.variable_scope('pooling1_lrn') as scope:
        pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME', name='pooling1')
        norm1 = tf.nn.lrn(pool1, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75, name='norm1')
        
    # conv2
    with tf.variable_scope('conv2') as scope:
        weights = tf.get_variable('weights',
                                  shape = [3, 3, 16, 16],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.1, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                  shape = [16],
                                  dtype = tf.float32,
                                  initializer = tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(norm1, weights, strides=[1,1,1,1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name='conv2')
        
    # pool2 and norm2
    with tf.variable_scope('pooling2_lrn') as scope:
        norm2 = tf.nn.lrn(conv2, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75, name='norm2')
        pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,1,1,1], padding='SAME', name='pooling2')
    
    with tf.variable_scope('dropout1') as scope:
        reshape = tf.reshape(pool2, shape=[batch_size, -1])
        dropout_layer1 = tf.nn.dropout(reshape, dropout_keep_prob, name = "dropout_layer1")
        
    # local3
    with tf.variable_scope('local3') as scope:
        weights = tf.get_variable('weights',
                                  shape = [173056, 128],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                  shape = [128],
                                  dtype = tf.float32,
                                  initializer = tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(dropout_layer1, weights) + biases, name=scope.name)
    
    with tf.variable_scope('dropout2') as scope:
        dropout_layer2 = tf.nn.dropout(local3, dropout_keep_prob, name = "dropout_layer2")
        
    # local4
    with tf.variable_scope('local4') as scope:
        weights = tf.get_variable('weights',
                                  shape = [128, 128],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                  shape = [128],
                                  dtype = tf.float32,
                                  initializer = tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(dropout_layer2, weights) + biases, name='local4')
        
    # softmax
    with tf.variable_scope('softmax_linear') as scope:
        weights = tf.get_variable('softmax_linear',
                                  shape = [128, n_classes],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                  shape = [n_classes],
                                  dtype = tf.float32,
                                  initializer = tf.constant_initializer(0.1))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name='softmax_linear')
        
    return softmax_linear

### 3.3 计算图

In [ ]:
tf.reset_default_graph()
train_graph = tf.Graph()
with train_graph.as_default():
    image = tf.placeholder(dtype=tf.float32, shape=[None, IMG_W, IMG_H, 3], name="image")
    label = tf.placeholder(dtype=tf.int32, shape=[None], name="label")
    LearningRate = tf.placeholder(tf.float32, name="LearningRate")
    
    with tf.variable_scope("inference"):
        softmax_linear = inference(image, BATCH_SIZE, N_CLASSES)
    
    with tf.variable_scope("loss"):
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=softmax_linear, labels=label)
        loss = tf.reduce_mean(cross_entropy, name='loss')
    
    with tf.variable_scope("optimizer"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step=global_step)
    
    with tf.variable_scope("accuracy"):
        correct = tf.nn.in_top_k(softmax_linear, label, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)

### 3.4 取得batch

In [ ]:
def get_batches(image, label, batch_size):
    for start in range(0, len(label), batch_size):
        end = min(start + batch_size, len(label))
        yield image[start:end], label[start:end]

## 四、训练和预测

In [ ]:
losses = {'train':[], 'cv':[]}
accurates = {'train':[], 'cv':[], 'test':[]}
with tf.Session(graph=train_graph) as sess:
    # Summaries for loss and accuracy
    loss_summary = tf.summary.scalar("loss", loss)
    accuracy_summary = tf.summary.scalar("accuracy", accuracy)
    
    # Train Summaries
    train_summary_op = tf.summary.merge([loss_summary, accuracy_summary])
    train_summary_writer = tf.summary.FileWriter("runs/train", sess.graph)
    
    # CV Summaries
    cv_summary_op = tf.summary.merge([loss_summary, accuracy_summary])
    cv_summary_writer = tf.summary.FileWriter("runs/cv", sess.graph)
    
    # Test Summaries
    test_summary_op = tf.summary.merge([loss_summary, accuracy_summary])
    test_summary_writer = tf.summary.FileWriter("runs/test", sess.graph)
    
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    for epoch_i in range(EPOCH_NUMS):
        train_X, cv_X, train_y, cv_y = train_test_split(train, train_label, test_size = 0.05)
        
        train_batches = get_batches(train_X, train_y, BATCH_SIZE)
        for batch_i in range(len(train_X) // BATCH_SIZE):
            x, y = next(train_batches)
            img = np.zeros([BATCH_SIZE, IMG_W, IMG_H, 3])
            for i in range(BATCH_SIZE):
                im = cv2.imread(x[i])
                if (im != None):
                    im = cv2.resize(im, (IMG_W, IMG_H))
                    img[i]=im
            
            feed = {image: img,
                    label: np.reshape(y, [BATCH_SIZE]),
                    LearningRate: learning_rate}
            step, train_loss, summaries, _, accurate = sess.run([global_step, loss, train_summary_op, train_op, accuracy], feed)  #cost
            losses['train'].append(train_loss)
            accurates['train'].append(accurate)
            train_summary_writer.add_summary(summaries, step)
            
            if (epoch_i * (len(train_X) // BATCH_SIZE) + batch_i) % 100 == 0:
                time_str = datetime.datetime.now().isoformat()
                print('{}: Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}    accuracy = {:.3f}'.format(time_str,
                                                                                                         epoch_i,
                                                                                                         batch_i,
                                                                                                         (len(train_X) // BATCH_SIZE),
                                                                                                         train_loss,
                                                                                                         accurate))
        cv_batches = get_batches(cv_X, cv_y, BATCH_SIZE)
        for batch_i in range(len(cv_X) // BATCH_SIZE):
            x, y = next(cv_batches)
            img = np.zeros([BATCH_SIZE, IMG_W, IMG_H, 3])
            for i in range(BATCH_SIZE):
                im = cv2.imread(x[i])
                if (im != None):
                    im = cv2.resize(im, (IMG_W, IMG_H))
                    img[i]=im
            
            feed = {image: img,
                    label: np.reshape(y, [BATCH_SIZE]),
                    LearningRate: learning_rate}
            step, cv_loss, summaries, accurate = sess.run([global_step, loss, cv_summary_op, accuracy], feed)  #cost
            losses['cv'].append(cv_loss)
            accurates['cv'].append(accurate)
            cv_summary_writer.add_summary(summaries, step)
            
            if (epoch_i * (len(cv_X) // BATCH_SIZE) + batch_i) % 20 == 0:
                time_str = datetime.datetime.now().isoformat()
                print('{}: Epoch {:>3} Batch {:>4}/{}   test_loss = {:.3f}    accuracy = {:.3f}'.format(time_str,
                                                                                                        epoch_i,
                                                                                                        batch_i,
                                                                                                        (len(cv_X) // BATCH_SIZE),
                                                                                                        cv_loss,
                                                                                                        accurate))
    test_batches = get_batches(test, test_label, BATCH_SIZE)
    for batch_i in range(len(test) // BATCH_SIZE):
        x, y = next(test_batches)
        img = np.zeros([BATCH_SIZE, IMG_W, IMG_H, 3])
        for i in range(BATCH_SIZE):
            im = cv2.imread(x[i])
            if (im != None):
                im = cv2.resize(im, (IMG_W, IMG_H))
                img[i]=im
        feed = {image: img,
                label: np.reshape(y, [BATCH_SIZE]),
                LearningRate: learning_rate}
        step, summaries, accurate = sess.run([global_step, test_summary_op, accuracy], feed)  #cost
        accurates['test'].append(accurate)
        test_summary_writer.add_summary(summaries, step)
    print("accuracy = {:.3f}".format(np.mean(accurates['test'])))
    saver.save(sess, 'save/save')
    print('Model Trained and Saved')